In [3]:

import numpy as np
import seaborn as sns
import pandas as pd

AttributeError: type object 'DeferredError' has no attribute 'new'

<font size='4' face='Times new Roman'>
    Dit worden de US Factors
    <font>

In [7]:

file_url = 'https://raw.githubusercontent.com/Sebasleen/Seminargroup/Seminar/US%20Factors/anomalies.dta'

# Read the Stata file content
Anomalies = pd.read_stata(file_url)

# Display unique values in the 'anomaly' column
print(Anomalies['anomaly'].unique())

# Display the DataFrame
print(Anomalies)

#Hier heb ik de global factors verwijderd
column_name = 'anomaly'
values_to_drop = ['glbab', 'glcma', 'glhml', 'glqmj', 'glrmw', 'glsmb', 'glumd']
mask = Anomalies[column_name].isin(values_to_drop)
Anomalies_US = Anomalies[~mask]

#Dit zijn dus alle 15 factors die alleen bij de US horen. 
print(Anomalies_US)





['ac' 'bab' 'cfp' 'cma' 'ep' 'hml' 'liq' 'ltrev' 'nsi' 'qmj' 'rmw' 'rvar'
 'smb' 'strev' 'umd' 'glbab' 'glcma' 'glhml' 'glqmj' 'glrmw' 'glsmb'
 'glumd']
       year  month anomaly    ret   time  global
0      1963      7      ac  2.170   42.0     0.0
1      1963      8      ac -0.197   43.0     0.0
2      1963      9      ac  0.600   44.0     0.0
3      1963     10      ac  6.463   45.0     0.0
4      1963     11      ac -2.260   46.0     0.0
...     ...    ...     ...    ...    ...     ...
12638  2019      8   glumd  2.990  715.0     1.0
12639  2019      9   glumd -3.260  716.0     1.0
12640  2019     10   glumd -0.940  717.0     1.0
12641  2019     11   glumd  0.000  718.0     1.0
12642  2019     12   glumd  0.740  719.0     1.0

[12643 rows x 6 columns]
       year  month anomaly    ret   time  global
0      1963      7      ac  2.170   42.0     0.0
1      1963      8      ac -0.197   43.0     0.0
2      1963      9      ac  0.600   44.0     0.0
3      1963     10      ac  6.463   4

<font size='4' face='Times new Roman'>    

Voor de duidelijkheid, dit zijn de US factors:

- **-ac**: Accruals
- **-bab**: Betting against Beta
- **-cfp**: Cash-flow to price
- **-cma**: Investment (Conservative minus aggressive)
- **-ep**: Earnings to price
- **-hml**: Value
- **-liq**: Liquidity
- **-ltrev**: Long-term reversal
- **-nsi**: Net share issue
- **-qmj**: Quality minus junk
- **-rmw**: Profitability
- **-rvar**: Residual Variance
- **-smb**: Size
- **-strev**: Short-term reversals
- **-umd**: Momentum

En dit zijn de global factors die ik zojuist heb gedropt:

- **glbab**: Betting against beta
- **glcma**: Investment (Conservative minus aggressive)
- **glhml**: Value (high minus low)
- **glqmj**: Quality minus junk
- **glrmw**: Profitability (robust minus weak)
- **glumd**: Momentum (up minus down)
- **glsmb**: Size (small minus big)

</font>


In [122]:
# Assuming df is your DataFrame
# Filter the DataFrame to include only rows where the 'anomaly' column has the value "smb"
filtered_df = Anomalies_US[Anomalies_US['anomaly'] == 'ac']

print(filtered_df)
# Calculate the mean of the 'ret' column for the filtered DataFrame
mean_ret_smb = filtered_df['ret'].mean()*10

# Print the mean value
print("Mean of 'ret' column for 'anomaly' values equal to 'smb':", mean_ret_smb)




     year  month anomaly    ret   time  global
0    1963      7      ac  2.170   42.0     0.0
1    1963      8      ac -0.197   43.0     0.0
2    1963      9      ac  0.600   44.0     0.0
3    1963     10      ac  6.463   45.0     0.0
4    1963     11      ac -2.260   46.0     0.0
..    ...    ...     ...    ...    ...     ...
673  2019      8      ac  1.850  715.0     0.0
674  2019      9      ac -1.473  716.0     0.0
675  2019     10      ac  0.020  717.0     0.0
676  2019     11      ac -0.250  718.0     0.0
677  2019     12      ac  0.120  719.0     0.0

[678 rows x 6 columns]
Mean of 'ret' column for 'anomaly' values equal to 'smb': 2.343997061252594


In [8]:
print(Anomalies_US)

       year  month anomaly    ret   time  global
0      1963      7      ac  2.170   42.0     0.0
1      1963      8      ac -0.197   43.0     0.0
2      1963      9      ac  0.600   44.0     0.0
3      1963     10      ac  6.463   45.0     0.0
4      1963     11      ac -2.260   46.0     0.0
...     ...    ...     ...    ...    ...     ...
10111  2019      8     umd  7.600  715.0     0.0
10112  2019      9     umd -6.850  716.0     0.0
10113  2019     10     umd  0.240  717.0     0.0
10114  2019     11     umd -2.620  718.0     0.0
10115  2019     12     umd -2.130  719.0     0.0

[10116 rows x 6 columns]
